In [1]:
using Pkg
using Random
Random.seed!(1234) ##settando a seed de números aleatórios
Pkg.add("DataFrames")
Pkg.add("Combinatorics")
using DataFrames
using Combinatorics
using Distributed

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


┌ Info: Recompiling stale cache file /home/matheus/.julia/compiled/v1.0/DataFrames/AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1190


# Capítulo 1

# Exercício 8

Usando o método descrito no exercício anterior, escreva um programa para determinar uma solução inteira para a equação $ax + by = c$, tendo como entrada os coeficientes $a$, $b$ e $c$. A saída do programa deve ser, ou uma solução inteira da equação, ou uma mensagem indicando que tais soluções não existem. Portanto o programa consistirá, essencialmente, de uma implementação do algoritmo euclidiano extendido.

In [2]:
function div(a,b) #sem usar a divisão e resto implementada pelo computador
    if b == 0
        throw(DivideError)
    elseif a == 0
        return 0
    end
    n = 1
    while n * b < a
        n = n * 2
    end
    while n * b > a
        n -= 1
    end
    return n
end

div (generic function with 1 method)

In [3]:
div(1234,1234)

1

In [4]:
div(1,12)

0

In [5]:
div(10,0)

DataType: DivideError

In [6]:
div(110,109)

1

In [7]:
function mdc(a,b)
    if a == 1
        return 1
    elseif b == 1
        return 1
    elseif a == 0
        return b
    elseif b == 0
        return a
    else
        A = max(a,b)
        B = min(a,b)
        Q = div(A,B) #quis implementar um algoritmo de divisão que não fosse preguiçoso
        if Q == 0
            return B
        else
            return mdc(B,A - B * Q)
        end
    end
end

mdc (generic function with 1 method)

In [8]:
@time mdc(1234,54)

  0.021792 seconds (8.02 k allocations: 393.895 KiB, 47.03% gc time)


2

In [9]:
function eext_aux(r1,r2,x1, x2, y1, y2)
    #queremos achar q e r tais que r= r2 + r1 q onde 0<=rj<r1
    qj = div(r2,r1)
    rj = r2 - r1 * qj
    if rj == 0
        return [x1,y1]
    else
        begin
        xj = x2 - qj * x1
        yj = y2 - qj * y1
        return eext_aux(rj,r1,xj,x1,yj,y1)
        end
    end
end

eext_aux (generic function with 1 method)

In [10]:
function euclidiano_extendido(a,b,c)
    if c % mdc(a,b) != 0
        return "Não tem solução"
    elseif a > b
        return eext_aux(a,b,0,1,1,0)
    else
        return eext_aux(b,a,0,1,1,0)
    end
end

euclidiano_extendido (generic function with 1 method)

In [11]:
@time euclidiano_extendido(1234,54,2)

  0.027565 seconds (33.99 k allocations: 1.759 MiB)


2-element Array{Int64,1}:
 160
  -7

## Exercício 9

Escreva um programa que, tendo como entrada dois inteiro $a$ e $b$, determina o máximo divisor comum de $a$ e $b$. Adapte seu programa para gerar aleatoriamente pares de inteiros $a$ e $b$ e calcular o $\mbox{mdc}(a,b)$. O programa deve ter como entrada o número total de pares que você deseja testar e, como saída o quociente $$\frac{\mbox{total de pares cujo mdc é }1}{\mbox{total de pares testados}}.$$

Este quociente dá uma medida da probabilidae de que um par de inteiros escolhido aleatoriamente seja co-primo. Vai ser necessário deixar o programa testar um número muito grande de pares para obter uma boa aproximação da probabilidade. Faça uma tabela com esses valores, tendo como entrada $10$, $100$, $1000$, $10000$ e $1000000$ pelo menos. Usando algoritmos de teoria da probabilidade que, testando um grande número de pares, o quociente acima deve ficar próximo de $\frac{6}{\pi^2}$; veja \[**Giblin 1993, p.16**\]. Como é que esse valor se compara com os resultados experimentais que você obteve?

Vamos utilizar a função de $\mbox{mmdc}$ que já implementamos. O gerador de números aleatórios poderá selecionar qualquer número interio.

In [12]:
rand(Int32,1,2)

1×2 Array{Int32,2}:
 -2113838868  -131651945

In [13]:
function exercicio9(n)
    A = rand(Int32,n)
    B = rand(Int32,n)
    return sum([1 for i = 1:n if mdc(max(A[i],-A[i]),max(-B[i],B[i])) == 1])/n
end

exercicio9 (generic function with 1 method)

In [14]:
function exercicio9()
for i = 1:6
    print(string(10^i) * ": " * string(exercicio9(10^i)) * "\n")
end
print("inf: " * string(6/pi^2) *"\n")
end
@time exercicio9()

10: 0.6
100: 0.61
1000: 0.598
10000: 0.6066
100000: 0.60604
1000000: 0.608326
inf: 0.6079271018540267
  0.740350 seconds (276.16 k allocations: 32.456 MiB, 1.22% gc time)


## Capítulo 2

 ## Exercício 11
 Seja $n$ um inteiro positivo e $d(n)$ o número de divisores positivos de $n$. Dizemos que $n$ é altamente composto se $d(m)<d(n)$ para todo $m<n$. Escreva um programa que, tendo como entrada um número inteiro $r$, determina todos os números altamente compostos menores que $r$. Use o programa para fazer uma lista dos números altamente compostos menores que $5000$. Observe as fatorações dos números altamente compostos da sua lista: que propriedades destes números você deduz da observação desta lista? Estes números foram estudados pelo matemático indiano autodidata Srinivasa Ramanujan.

In [15]:
function primos_aux(n, lista, div)
    n = Int(n)
    if n == 1 
        return lista
    elseif div==n # n eh primo
          return push!(lista,n)
    elseif n % div == 0 # div divide n
        return primos_aux(n/div,push!(lista,div), div)
    elseif div^2>n
        return lista
    else
        return primos_aux(n, lista, div + 2)
    end
end

primos_aux (generic function with 1 method)

In [16]:
function primos(n)
    lista = [] # sorry for not be trully functional D:
    while n % 2 == 0 # it hurts more in me than in you
        push!(lista, 2) # but, performance, right?
        n = Int(n/2)
    end
    return if (x = primos_aux(n,lista,3)) != [] x else [n] end
end

primos (generic function with 1 method)

In [17]:
function divisores_aux(esperando, resp)
    if size(esperando)[1] > 0
        return divisores_aux(
            esperando[1:end-1],
                vcat(resp,
                     filter(
                        x -> !(x in resp),
                        map(x -> x * esperando[end], resp)))) 
    else
        return resp
    end
end
    
function divisores(n)
    return sort(divisores_aux(primos(n), [1]))
end

divisores (generic function with 1 method)

In [18]:
d(n) = size(divisores(n))[1]

d (generic function with 1 method)

In [19]:
e_altamente_composto(r) = eac_aux(r,r-1)

function eac_aux(n,m)
    if m == 1
        true
    elseif d(m) < d(n)
        eac_aux(n,m-1)
    else
        false
    end
end

eac_aux (generic function with 1 method)

In [20]:
e_altamente_composto(20)

false

In [21]:
todos_altamente_compostos(r) = [i for i = 2:r if e_altamente_composto(i)]

todos_altamente_compostos (generic function with 1 method)

In [22]:
@time todos_altamente_compostos(5000)

  0.559293 seconds (3.53 M allocations: 231.155 MiB, 12.57% gc time)


18-element Array{Int64,1}:
    2
    4
    6
   12
   24
   36
   48
   72
  144
  216
  288
  432
  576
  600
  900
 1200
 1800
 3600

##  Exercícico 12

Escreva um programa que impelemente o algoritmo de Fermat para achar dois fatores de um número inteiro positivo que seja menor que $2^{32}$. Este exercício é o primeiro de uma sequência que termina com o exercício 8 do capítulo 1.

In [23]:
function fermat(n)
    x = floor(sqrt(n))
    if x^2 == n
        return (x,x)
    else
        x += 1
        y = sqrt(x^2 - n)
        while !isinteger(y)
            x += 1
            y = sqrt(x^2 - n)
            if x == (n+1)/2
                return string(n) * " é primo"
            end
        end
        return (Int(x+y),Int(x-y))
    end
end

fermat (generic function with 1 method)

In [24]:
@time fermat(1342127)

  0.032219 seconds (30.46 k allocations: 1.542 MiB)


(1277, 1051)

 # Capítulo 3
 
 ## Exercício 10
 
 O objetivo deste exercício é a elaboração de um programa para determinar alguns valores (interios) de $x$ para os quais um polinômio quadrático $f(x)$ assume valores primos. O programa deve ter como entrada $f(x) = ax^2 + bx +c$. Estes números serão inteiros positivos ou negativos, mas podemos supor que $a>0$. O programa deve tabelar $f(n)$, para $n$ variando entre $0$ e $100$, e indicar quais destes valores são primos. Para fazer isso, será necessário executar o crivo de Erastótenes para achar os primos menores que $|f(0)|$ ou $|f(100|$ - dependendo de qual dos dois for maior. Observe que vai ser necessário limitar os valores de $a$, $b$ e $c$, e verificar se os valores de $f(x)$ não estão ultrapassando a precisão dos inteiros da linguagem de programação que está sendo utilizada. Se você não tomar estes cuidados o programa vai produzir resultados incorretos. Aplique seu programa para os polinômios 
 1. $f(x) = x^2 + 1$
 2. $f(x) = x^2 - 69x + 1231$
 3. $f(x) = 2x^2 - 199$
 4. $f(x) = 8x^2 - 530x + 7631$
 
 O polinômio (2) é uma variação de um famoso exemplo que L. Euler obteve em 1772.

In [25]:
function eras_aux(lista, iter)
    len = size(lista)[1]
    if  iter >= len
        return lista
    elseif lista[iter] == 0
        return eras_aux(lista,iter+2)
    else
        step = iter
        iter += step
        while iter <= len
            lista[iter] = 0
            iter += step
        end
        return eras_aux(lista,step+2)
    end
end

function erastotenes(n)
    lista = [if i%2==0 0 else 1 end for i=1:n]
    lista[1] = 0
    return [x[1] for x in enumerate(eras_aux(lista,3)) if x[2] == 1 || x[1] == 2]
end    

erastotenes (generic function with 1 method)

In [26]:
erastotenes(100)

25-element Array{Int64,1}:
  2
  3
  5
  7
 11
 13
 17
 19
 23
 29
 31
 37
 41
 43
 47
 53
 59
 61
 67
 71
 73
 79
 83
 89
 97

In [27]:
function exercicio10(a,b,c)
    f(n) = a * n^2 + b * n + c
    primos = erastotenes(max(abs(f(0)),abs(f(100))))
    return [(n, f(n),if f(n) in primos "é primo" else "não primo" end) for n=1:100]
end

exercicio10 (generic function with 1 method)

In [28]:
@time exercicio10(1,0,1)

  0.123347 seconds (118.28 k allocations: 5.881 MiB)


100-element Array{Tuple{Int64,Int64,String},1}:
 (1, 2, "é primo")        
 (2, 5, "é primo")        
 (3, 10, "não primo")     
 (4, 17, "é primo")       
 (5, 26, "não primo")     
 (6, 37, "é primo")       
 (7, 50, "não primo")     
 (8, 65, "não primo")     
 (9, 82, "não primo")     
 (10, 101, "é primo")     
 (11, 122, "não primo")   
 (12, 145, "não primo")   
 (13, 170, "não primo")   
 ⋮                        
 (89, 7922, "não primo")  
 (90, 8101, "é primo")    
 (91, 8282, "não primo")  
 (92, 8465, "não primo")  
 (93, 8650, "não primo")  
 (94, 8837, "é primo")    
 (95, 9026, "não primo")  
 (96, 9217, "não primo")  
 (97, 9410, "não primo")  
 (98, 9605, "não primo")  
 (99, 9802, "não primo")  
 (100, 10001, "não primo")

In [29]:
@time exercicio10(1,-69,1231)

  0.000112 seconds (123 allocations: 54.766 KiB)


100-element Array{Tuple{Int64,Int64,String},1}:
 (1, 1163, "é primo")    
 (2, 1097, "é primo")    
 (3, 1033, "é primo")    
 (4, 971, "é primo")     
 (5, 911, "é primo")     
 (6, 853, "é primo")     
 (7, 797, "é primo")     
 (8, 743, "é primo")     
 (9, 691, "é primo")     
 (10, 641, "é primo")    
 (11, 593, "é primo")    
 (12, 547, "é primo")    
 (13, 503, "é primo")    
 ⋮                       
 (89, 3011, "é primo")   
 (90, 3121, "é primo")   
 (91, 3233, "não primo") 
 (92, 3347, "é primo")   
 (93, 3463, "é primo")   
 (94, 3581, "é primo")   
 (95, 3701, "é primo")   
 (96, 3823, "é primo")   
 (97, 3947, "é primo")   
 (98, 4073, "é primo")   
 (99, 4201, "é primo")   
 (100, 4331, "não primo")

In [30]:
@time exercicio10(2,0,-199)

  0.000486 seconds (125 allocations: 223.766 KiB)


100-element Array{Tuple{Int64,Int64,String},1}:
 (1, -197, "não primo")  
 (2, -191, "não primo")  
 (3, -181, "não primo")  
 (4, -167, "não primo")  
 (5, -149, "não primo")  
 (6, -127, "não primo")  
 (7, -101, "não primo")  
 (8, -71, "não primo")   
 (9, -37, "não primo")   
 (10, 1, "não primo")    
 (11, 43, "é primo")     
 (12, 89, "é primo")     
 (13, 139, "é primo")    
 ⋮                       
 (89, 15643, "é primo")  
 (90, 16001, "é primo")  
 (91, 16363, "é primo")  
 (92, 16729, "é primo")  
 (93, 17099, "é primo")  
 (94, 17473, "não primo")
 (95, 17851, "é primo")  
 (96, 18233, "é primo")  
 (97, 18619, "não primo")
 (98, 19009, "é primo")  
 (99, 19403, "é primo")  
 (100, 19801, "é primo") 

In [31]:
@time exercicio10(8,-530,7681)

  0.000885 seconds (125 allocations: 340.016 KiB)


100-element Array{Tuple{Int64,Int64,String},1}:
 (1, 7159, "é primo")     
 (2, 6653, "é primo")     
 (3, 6163, "é primo")     
 (4, 5689, "é primo")     
 (5, 5231, "é primo")     
 (6, 4789, "é primo")     
 (7, 4363, "é primo")     
 (8, 3953, "não primo")   
 (9, 3559, "é primo")     
 (10, 3181, "é primo")    
 (11, 2819, "é primo")    
 (12, 2473, "é primo")    
 (13, 2143, "é primo")    
 ⋮                        
 (89, 23879, "é primo")   
 (90, 24781, "é primo")   
 (91, 25699, "não primo") 
 (92, 26633, "é primo")   
 (93, 27583, "é primo")   
 (94, 28549, "é primo")   
 (95, 29531, "é primo")   
 (96, 30529, "é primo")   
 (97, 31543, "é primo")   
 (98, 32573, "é primo")   
 (99, 33619, "é primo")   
 (100, 34681, "não primo")

 ## Exercício 11
 
 Vimos na seção 5 que existem várias formas que dão aproximações para $\pi(x)$, o número de primos positivos menores ou iguais a $x$. A fórmula decorrente do teorema dos números primos é $\frac{x}{\log x}$ que não fornece uma boa aproximação a não ser que o valor de $x$ seja enorme. Neste exercício desejamos fazer o estudo experimental de uma outra forma que serve de aproximação para $\pi(x)$. A fórmula é $$ S(x)=\frac{x}{\log x} \left(1+\left[\sum_{k=0}^{12} a_k\left(\log \log x\right)^k\right]^{-\frac{1}{4}}\right) $$
 onde $\log$ denota o *logaritmo natural* e $$a_0 = 229168.50747390,\ a_1 = -429449.7206839,\ a_2=199330.41355048,\ a_3=28226.22049280,\\ a_4 = 0,\ a_5=0,\ a_6 = -34712.81875914,\ a_7=0,\ a_8 = 33820.10886195,\\ a_9=-25379.82656589,\ a_{10}=8386.14942934,\ a_{11} = -1360.44512548,\ a_{12}=89.14545378. $$
 
Escreva um programa, baseado no *Crivo de Erastótenes* que, tendo como entrada o inteiro $x$, calcula $\pi(x)$. Use este algoritmo para gerar uma tabela com os valores de $\pi(x) - S(x)$ quando $x$ é igual a $11$, $100$, $1000$, $2000$, $3000$, $\dots$, $9000$ e $10000$. Comare com os valores correspondentes de $\pi(X) - \frac{x}{\log x}$. O que você conclui da análise destas tabelas?

In [32]:
function S(x) 
    a = [229168.50747390,
        -429449.7206839,
        199330.41355048,
        28226.22049280,
        0,
        0,
        -34712.81875914,
        0,
        33820.10886195,
        -25379.82656589,
        8386.14942934,
        -1360.44512548,
        89.14545378]
    return (x/log(x)) * (1 + sum([a[k+1] * (log(log(x)))^k for k=0:12])^(-1/4))
end

S (generic function with 1 method)

In [33]:
Π(x) = size(erastotenes(x))[1]

Π (generic function with 1 method)

In [34]:
defaultaprox(x) = x\log(x)

defaultaprox (generic function with 1 method)

In [35]:
function exercicio11()
    values = [11, 100, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
    pilist = [Π(x) for x in values]
    slist = [S(x) for x in values]
    deflist = [defaultaprox(x) for x in values]
    PiSMinus = [pilist[i[1]]-slist[i[1]] for i in enumerate(values)]
    PiDefMinus = [pilist[i[1]]-deflist[i[1]] for i in enumerate(values)]
    DataFrame(value=values, pi_minus_s=PiSMinus, pi_minus_default = PiDefMinus)
end
@time exercicio11()

  1.482386 seconds (2.24 M allocations: 113.171 MiB, 4.77% gc time)


,value,pi_minus_s,pi_minus_default
,Int64,Float64,Float64
1,11,7.7153e-7,4.78201
2,100,-0.00139032,24.9539
3,1000,0.106508,167.993
4,2000,-1.10818,302.996
5,3000,-1.30125,429.997
6,4000,-3.18811,549.998
7,5000,-2.4107,668.998
8,6000,-3.87482,782.999
9,7000,-0.146994,899.999


Eu concluo que a formula Default aumenta o erro conforme o valor aumenta, enquanto a fórmula $S$ mágica, mantem o erro estável.

## Exercício 12

Já vimos que um primo ímpar ou é da forma $4n+1$ ou da forma $4n+3$. Além disso, sabemos pelo exercício 7 que há infinitos primos da forma $4n+3$. Também há infinitos primos da forma $4n+1$, mas a demonstração é mais difícil. O objetivo deste exercício é estudar experimentalmente a frequência relativa de cada um destes dois tipos de primos. Seja $x$ um número real positivo. Denotaremos por $\pi_1(x)$ o número de primos positivos que são da forma $4n+1$; e por $\pi_3(x)$ o número correspondente de primos da forma $4n+3$. Escreva um programa, baseado no *Crivo de Erastótenes*, que, tendo como entrada um inteiro positivo $x$, determina $\pi_1(x)$ e $\pi_3(x)$. Tabele $\pi_1(x)$, $\pi_3(x)$ e $\frac{\pi_1(x)}{\pi_3(x)}$ para os valores de $100$ em $100$ entre $100$ e $10^5$. Sabe-se que $\lim_{x\rightarrow \infty} \frac{\pi_1(x)}{\pi_3(x)} = 1$. Os valores da sua tabela condizem com este resultado?

In [36]:
function is_4n_mais_1(x)
    return (x-1)%4 == 0
end

function exercicio12(x) 
    primos = erastotenes(x)[2:end]  #queremos descartar o 2, o primeiro primo
    pi1 = [x for x in primos if is_4n_mais_1(x)]
    return (size(pi1)[1], size(primos)[1] - size(pi1)[1]) #retorna o pi1 e total de primos menos o pi1, que é o pi3
end

exercicio12 (generic function with 1 method)

In [37]:
exercicio12(100)

(11, 13)

In [38]:
function exercicio12()
    Π1 = []
    Π3 = []
    ΠDiv = []
    values = []
    for num = 100:100:10^5
        π1, π3 = exercicio12(num)
        push!(values,num)
        push!(Π1,π1)
        push!(Π3,π3)
        push!(ΠDiv,π1/π3)
    end
    showall(DataFrame(value = values, Π1 = Π1, Π3 = Π3, ΠDiv = ΠDiv))
end
@time exercicio12()

┌ Warning: `showall(df::AbstractDataFrame, allcols::Bool=true)` is deprecated, use `show(df, allrows=true, allcols=allcols)` instead.
│   caller = showall(::DataFrame) at deprecated.jl:54
└ @ DataFrames ./deprecated.jl:54


1000×4 DataFrame
│ Row  │ value  │ Π1   │ Π3   │ ΠDiv     │
│      │ Any    │ Any  │ Any  │ Any      │
├──────┼────────┼──────┼──────┼──────────┤
│ 1    │ 100    │ 11   │ 13   │ 0.846154 │
│ 2    │ 200    │ 21   │ 24   │ 0.875    │
│ 3    │ 300    │ 29   │ 32   │ 0.90625  │
│ 4    │ 400    │ 37   │ 40   │ 0.925    │
│ 5    │ 500    │ 44   │ 50   │ 0.88     │
│ 6    │ 600    │ 51   │ 57   │ 0.894737 │
│ 7    │ 700    │ 59   │ 65   │ 0.907692 │
│ 8    │ 800    │ 67   │ 71   │ 0.943662 │
│ 9    │ 900    │ 74   │ 79   │ 0.936709 │
│ 10   │ 1000   │ 80   │ 87   │ 0.91954  │
│ 11   │ 1100   │ 89   │ 94   │ 0.946809 │
│ 12   │ 1200   │ 95   │ 100  │ 0.95     │
│ 13   │ 1300   │ 104  │ 106  │ 0.981132 │
│ 14   │ 1400   │ 109  │ 112  │ 0.973214 │
│ 15   │ 1500   │ 116  │ 122  │ 0.95082  │
│ 16   │ 1600   │ 119  │ 131  │ 0.908397 │
│ 17   │ 1700   │ 128  │ 137  │ 0.934307 │
│ 18   │ 1800   │ 135  │ 142  │ 0.950704 │
│ 19   │ 1900   │ 140  │ 149  │ 0.939597 │
│ 20   │ 2000   │ 147  │ 155  │ 0.948

│ 252  │ 25200  │ 1383 │ 1397 │ 0.989979 │
│ 253  │ 25300  │ 1387 │ 1400 │ 0.990714 │
│ 254  │ 25400  │ 1393 │ 1406 │ 0.990754 │
│ 255  │ 25500  │ 1397 │ 1412 │ 0.989377 │
│ 256  │ 25600  │ 1402 │ 1415 │ 0.990813 │
│ 257  │ 25700  │ 1409 │ 1420 │ 0.992254 │
│ 258  │ 25800  │ 1413 │ 1426 │ 0.990884 │
│ 259  │ 25900  │ 1418 │ 1429 │ 0.992302 │
│ 260  │ 26000  │ 1423 │ 1436 │ 0.990947 │
│ 261  │ 26100  │ 1428 │ 1439 │ 0.992356 │
│ 262  │ 26200  │ 1434 │ 1444 │ 0.993075 │
│ 263  │ 26300  │ 1440 │ 1449 │ 0.993789 │
│ 264  │ 26400  │ 1445 │ 1454 │ 0.99381  │
│ 265  │ 26500  │ 1450 │ 1459 │ 0.993831 │
│ 266  │ 26600  │ 1456 │ 1461 │ 0.996578 │
│ 267  │ 26700  │ 1461 │ 1466 │ 0.996589 │
│ 268  │ 26800  │ 1467 │ 1471 │ 0.997281 │
│ 269  │ 26900  │ 1475 │ 1475 │ 1.0      │
│ 270  │ 27000  │ 1479 │ 1481 │ 0.99865  │
│ 271  │ 27100  │ 1483 │ 1487 │ 0.99731  │
│ 272  │ 27200  │ 1485 │ 1493 │ 0.994642 │
│ 273  │ 27300  │ 1489 │ 1499 │ 0.993329 │
│ 274  │ 27400  │ 1493 │ 1500 │ 0.995333 │
│ 275  │ 27

│ 624  │ 62400  │ 3120 │ 3145 │ 0.992051 │
│ 625  │ 62500  │ 3125 │ 3149 │ 0.992379 │
│ 626  │ 62600  │ 3130 │ 3153 │ 0.992705 │
│ 627  │ 62700  │ 3133 │ 3159 │ 0.99177  │
│ 628  │ 62800  │ 3137 │ 3163 │ 0.99178  │
│ 629  │ 62900  │ 3142 │ 3166 │ 0.992419 │
│ 630  │ 63000  │ 3147 │ 3172 │ 0.992119 │
│ 631  │ 63100  │ 3150 │ 3176 │ 0.991814 │
│ 632  │ 63200  │ 3153 │ 3181 │ 0.991198 │
│ 633  │ 63300  │ 3156 │ 3184 │ 0.991206 │
│ 634  │ 63400  │ 3164 │ 3189 │ 0.992161 │
│ 635  │ 63500  │ 3168 │ 3196 │ 0.991239 │
│ 636  │ 63600  │ 3173 │ 3200 │ 0.991563 │
│ 637  │ 63700  │ 3179 │ 3207 │ 0.991269 │
│ 638  │ 63800  │ 3185 │ 3212 │ 0.991594 │
│ 639  │ 63900  │ 3189 │ 3216 │ 0.991604 │
│ 640  │ 64000  │ 3195 │ 3217 │ 0.993161 │
│ 641  │ 64100  │ 3199 │ 3222 │ 0.992862 │
│ 642  │ 64200  │ 3203 │ 3226 │ 0.99287  │
│ 643  │ 64300  │ 3205 │ 3231 │ 0.991953 │
│ 644  │ 64400  │ 3209 │ 3235 │ 0.991963 │
│ 645  │ 64500  │ 3212 │ 3240 │ 0.991358 │
│ 646  │ 64600  │ 3215 │ 3243 │ 0.991366 │
│ 647  │ 64

│ 983  │ 98300  │ 4704 │ 4734 │ 0.993663 │
│ 984  │ 98400  │ 4709 │ 4738 │ 0.993879 │
│ 985  │ 98500  │ 4712 │ 4746 │ 0.992836 │
│ 986  │ 98600  │ 4716 │ 4750 │ 0.992842 │
│ 987  │ 98700  │ 4720 │ 4753 │ 0.993057 │
│ 988  │ 98800  │ 4725 │ 4756 │ 0.993482 │
│ 989  │ 98900  │ 4733 │ 4760 │ 0.994328 │
│ 990  │ 99000  │ 4738 │ 4766 │ 0.994125 │
│ 991  │ 99100  │ 4743 │ 4769 │ 0.994548 │
│ 992  │ 99200  │ 4749 │ 4774 │ 0.994763 │
│ 993  │ 99300  │ 4754 │ 4777 │ 0.995185 │
│ 994  │ 99400  │ 4758 │ 4781 │ 0.995189 │
│ 995  │ 99500  │ 4762 │ 4784 │ 0.995401 │
│ 996  │ 99600  │ 4765 │ 4790 │ 0.994781 │
│ 997  │ 99700  │ 4767 │ 4796 │ 0.993953 │
│ 998  │ 99800  │ 4773 │ 4800 │ 0.994375 │
│ 999  │ 99900  │ 4779 │ 4804 │ 0.994796 │
│ 1000 │ 100000 │ 4783 │ 4808 │ 0.9948   │  3.118052 seconds (4.21 M allocations: 789.236 MiB, 4.15% gc time)


**Sim**, os valores condizem.

## Exercício 13

Adapte o programa elaborado no exercício anterior para determinar o menor valor inteiro de $x$ para o qual $\pi_1(x) > \pi_3(x)$. 

O exame da evidência experimental entõ disponível leou alguns matemáticos do começo do século \[sic: passado\] à conclusão de que, exceto para alguns valores relativamente pequenos de $x$, a desigualdade $\pi_1(x) < \pi_3(x)$ seria sempre verdadeira. A verdade emergiu em 1914, quando o matemático J. E. Littlewood mostrou que existem sequências infinitas $$x_1,x_2,\dots \ \mbox{e}\ y_1,y_2,\dots$$ de números reais positivos, tais que $$\lim_{i\rightarrow \infty} (\pi_1(x_i) -\pi_3(x_i)) = \infty \ \mbox{e}\ \lim_{i\rightarrow \infty} (\pi_1(y_i) -\pi_3(y_i)) = -\infty.$$

A moral é óbvia: é sempre muito perigoso generalizar a partir de dados numéricos, mesmo quando a quantidade de dados disponíveis é muito grande.

In [39]:
function exercicio13()
    x = 1
    π1, π3 = exercicio12(x) #
    while π1 <= π3
        x += 1
        π1, π3 = exercicio12(x)
    end
    return x
end

exercicio13 (generic function with 1 method)

In [40]:
@time exercicio13()

  6.203362 seconds (882.60 k allocations: 4.435 GiB, 5.01% gc time)


26861

# Capítulo 4

## Exercício 11

Escreva um programa que implemente o algoritmo para calcular potências módulo $n$ que é descrito na seção 2 do apêndice. O algoritmo deverá ter como entrada $a$, $k$ e $n$, onde $a$ é um inteiro qualquer $k$ e $n$ são inteiros positivos. A saída deverá ser a forma reduzida de $a^k$ módulo $n$. Este algoritmo é uma parte fundamental de quase todos os algoritmos que estudaremos a partir do capítulo 6.

### Algoritmo de exponenciação:

**Entrada:** *Inteiros $a$, $e$ e $n$, $n>0$ e $e\geq 0$*
**Saída:**  *a forma reduzida de $a^e$ módulo $n$.*

**Etapa 1:** *Comece com $A=a$, $P=1$ e $E=e$.* 

**Etapa 2:** *Se $E=0$ imprime '$a^e\equiv P\ (\mbox{mod}\ n)$'; se não vai para Etapa 3;*

**Etapa 3:** *Se $E$ for ímpar então atribua a $P$ o valor do resto da divisão de $A\cdot P$ por $n$ e a $E$ o valor $\frac{E-1}{2}$ e vá para a Etapa 5, se não vá para a Etapa 4;*

**Etapa 4:** *Se $E$ for par então atribua a $E$ o valor $\frac{E}{2}$ e vá para a Etapa 5.*

**Etapa 5:** *Substitua o valor atual de $A$ pelo resto da divisão de $A^2$ por $n$ e vá para a Etapa 2.*


In [41]:
function exercicio11(a,e,n) ##feito puramente funcional :P
    return verifica_pos(etapa2(a,1,e,n),n)
end

function etapa2(A,P,E,n)
    if E==0
        return P
    else
        return etapa3(A,P,E,n)
    end
end   

function etapa3(A,P,E,n)
    if E%2 == 1
        return etapa5(A,(A * P) % n,(E-1)/2,n)
    else
        return etapa4(A,P,E,n)
    end
end

function etapa4(A,P,E,n)
    if E%2 == 0  ##eu acho este passo muito estranho, mas o pseudo-código do livro pediu pra fazer assim e assim o fiz
        return etapa5(A,P,E/2,n)
    end
end

function etapa5(A,P,E,n)
    return etapa2(A^2 % n, P, E, n)
end

function verifica_pos(a,n) #função que garante que apenas iremos retornar resultados positivos
    if a>=0
        return a % n
    else #se for negativo
        return (n+a) % n #retorna o correspondente positivo
    end
end

verifica_pos (generic function with 1 method)

In [42]:
exercicio11(10,135,7) ##na pagina 79 do livro temos que esse valor é 6

6

In [43]:
exercicio11(5,3,13) ##retirado da Wikipedia, espera-se 8

8

In [44]:
@time exercicio11(4,13,497) ##tambem retirado da Wikipedia, espera-se 445

  0.000005 seconds (4 allocations: 160 bytes)


445

# Capítulo 5

## Exercício 16

Use o exercício 14 para escrever um programa que, tendo como entrada $a$ e $p$, calcula o inverso de $a$ módulo $p$. Por segurança faça seu programa verificar primeiro que $p$ *realmente* não divide $a$. Use isto para escrever um outro programa que determine a solução de $ax\equiv b\ (\mbox{mod}\ p)$, dada as entradas $a$, $b$ e $p$.

#### Exercício 14

Seja $p$ um número primo e $a$ um inteiro que não é divisível por $p$. Mostre que o invero de $\bar{a}$ em $\mathbb{Z}_p$ é $\bar{a}^{p-2}$.

In [45]:
function inverso(a,p)
    if p in erastotenes(p) && a % p != 0  ##verifica se p é primo e se p não divide a
        return ((a % p)^(p-2) % p)
    else
        throw(ArgumentError)
    end
end

inverso (generic function with 1 method)

In [46]:
inverso(2,5) #esperamos que seja 3

3

Note que $x\equiv ba^{-1}\ (\mbox{mod}\ p)$. Isso é tudo que precisamos

In [47]:
exercicio16(a,b,p) = (b * inverso(a,p)) % p

exercicio16 (generic function with 1 method)

Vamos testar nosso programa para $8x \equiv 3\ (\mbox{mod}\ 7)$. Esperamos que $x = 3$, porque $8\equiv 1\ (\mbox{mod}\ 7)$.

In [48]:
@time exercicio16(8,3,7)

  0.006385 seconds (1.19 k allocations: 65.152 KiB)


3

# Exercício 17

Seja $p = 4k+3$ um primo positivo. Escreva um programa que, tendo por entrada $p$ e um inteiro positivo $a$, calcula as duas soluções de $x^2\equiv a\ (\mbox{mod}\ p)$. Observe que sabemos do exercício 15 que *se esta equação tem solução* $b$ então $b \equiv a^{k+1}\ (\mbox{mod}\ p)$. Assim o programa deve calcular a forma reduzida de $a^{k+1}$ módulo $p$ e, em seguida, verificar se, de fato, esta é uma solução da equação dada. A saída será constituída pelas soluções da equação, ou por uma mensagem indicando que a equação dada não tem solução. Este exercício é o segundo de uma sequência que termna com o exercício 8 do capítulo 11.

In [49]:
function extrai_k(p)
    if p in erastotenes(p) && p % 4 == 3 #se é primo e de fato da forma 4k+3
        return (p-3)/4
    else
        throw(ArgumentError)
    end
end

function exercicio17(a,p)
    b = a^(extrai_k(p)+1) % p
    if b^2 % p == a
        return (b,-b)
    else
        return "Não tem solução"
    end
end

exercicio17 (generic function with 1 method)

Vamos testar com $x^2 \equiv 9\ (\mbox{mod}\ 11)$ que sabemos que tem que ser igual a $\pm 3$.

In [50]:
@time exercicio17(9,11)

  0.033080 seconds (26.58 k allocations: 1.363 MiB)


(3.0, -3.0)

## Exercício 18

Vários problemas da Teoria dos Números (como o Último Teorema de Fermat) requerem primos $p$ que satisfaçam a congruência $$a^{p-1} \equiv 1\ (\mbox{mod}\ p^2)$$ para algum inteiro $a$. Escreva um programa que, tendo por entrada dois inteiros positivos $a>1$ e $r$, determina os primos $p$ que satisfaçam a congruência acima e tais que $a+1\leq p \leq r$. Observe que o programa terá que aplicar o crivo de Eratóstenes para achar os primos menos ou iguais a $r$. Estes primos serão então testados na congruência acima. Se $r=10^5$ então o número de primos satisfazendo a congruência é dois quando $a=2,\ 5,\ 10$ e $14$; e cinco quando $a=19$, por exemplo.

In [51]:
## também feito puramente funcional :P
## a gente filta o erastotenes com uma expressão lambda (x) -> a+1 <= x
## também usamos a função exercício11 que serve para realizar a exponenciação de a^e (mod n).
exercicio18(a,r) = [p for p in filter((x) -> a+1 <= x , erastotenes(r)) if exercicio11(a,p-1,p^2) == 1] 

exercicio18 (generic function with 1 method)

In [52]:
@time exercicio18(5,10^5)

  0.090082 seconds (101.07 k allocations: 6.098 MiB)


2-element Array{Int64,1}:
 20771
 40487

In [53]:
@time exercicio18(10, 10^5) #TODO nao sei porque não funciona

  0.018912 seconds (44 allocations: 1.265 MiB)


1-element Array{Int64,1}:
 487

In [54]:
@time exercicio18(14, 10^5)

  0.019080 seconds (44 allocations: 1.265 MiB)


2-element Array{Int64,1}:
  29
 353

In [55]:
@time exercicio18(19,10^5) ##TODO não sei porque não funciona

  0.017976 seconds (44 allocations: 1.265 MiB)


2-element Array{Int64,1}:
  43
 137

Eu implementei em Python também, usando um Crivo de Eratóstenes que peguei no Rosetta Code (tentei adaptar o que escrevi aqui, mas o Python tilta na recursão) e usando a função pow do Python. Deu os mesmos resultados. Então não sei o que está pegando.

# Capitulo 6

## Exercício 8

Escreva um programa que, tendo como entrada um inteiro positivo $r$, determina todos os números entre $1$ e $r$ que sao pseudoprimos para as bases $2$ e $3$. Lembre-se que tal pseudoprimo $n$ é um número *ímpar* e *composto* que satisfaz as equações $$2^{n-1}\equiv 1\ (\mbox{mod}\ n)\ \mbox{e}\ 3^{n-1}\equiv 1\ (\mbox{mod}\ n).$$

Portanto o programa só precisa testar os números ímpares e além destas congruências, tem que verificar se os números obtidos são primos ou não. Uma maneira de fazer isso é programar o Crivo de Eratóstenes de modo a conservar a lista dos ímpares compostos, em vez dos primos e depois testar as congruências. Aplique o programa  com $r=10^5$. Quantos e quais foram os pseudoprimos obtidos? Quantos destes são números de Carmichael?

In [56]:
function e_pseudoprimo(num,base, primos)
    if !(num in primos && num % 2 == 1)
        return exercicio11(base,num-1,num)==1
    else
        return false
    end
end

e_pseudoprimo (generic function with 1 method)

In [57]:
function e_carmichael(n)
    if n in [1,2,3]
        return false
    end
    if n%2 == 1 && !(n in erastotenes(n))
        return reduce((x,y)-> x && y, [exercicio11(b,n,n) == b for b=2:(n-2)])
    else
        return false
    end
end

e_carmichael (generic function with 1 method)

In [58]:
e_carmichael(559)

false

In [59]:
[num for num=1:600 if e_carmichael(num)] ##todos números carmichael até 600

1-element Array{Int64,1}:
 561

In [60]:
function exercicio8(r) 
    primos = erastotenes(r)
    return [n for n=1:2:r if e_pseudoprimo(n,2,primos) && e_pseudoprimo(n,3,primos)]
end

exercicio8 (generic function with 1 method)

In [61]:
@time lista = exercicio8(10^5)

  0.520491 seconds (102.72 k allocations: 5.996 MiB)


23-element Array{Int64,1}:
  1105
  1729
  2465
  2701
  2821
  6601
  8911
 10585
 15841
 18721
 29341
 31621
 41041
 46657
 49141
 52633
 63973
 75361
 83333
 83665
 88561
 90751
 93961

In [62]:
print("Foram " * string(size(lista)[1]) * " pseudoprimos obtidos")

Foram 23 pseudoprimos obtidos

In [63]:
print("Temos " * string(size(filter((x)->e_carmichael(x),lista))[1]) * " números de Carmichael")

Temos 14 números de Carmichael

## Exercício 9

O objetivo deste exercício é a elaboração de um programa para determinar todos os números de Carmichael que são produtos de $d$ primos, todos menores que $10^3$. O principal problema é que, mesmo para valores relativamente pequenos de de $d$, alguns dos números de Carmichael obtidos podem ser bem grandes. Isto torna difícil verificar que $p-1$ divide $n-1$, quando $p$ é um fator primo de $n$. Mas é fácil contornar este problema usando congruências. Digamos que o programa gerou o número $n = p_1p_2\dots p_d$. Precisamos verificar se $n-1$ é ou não divisível por $p_1-1$, por exemplo. Como $n$ aparece fatorado, você pode multiplicar os fatores um de cada vez e ir reduzindo módulo $p_1-1$ a cada produto, de modo a manter o resultado pequeno. Como os fatores são menores que $10^3$ isto vai ser muito rápido de executar. Aplique o seu programa para $d=1,\dots,8$. Quais números de Carmichael obtidos em cada caso? Não é necessário multiplicar os fatores para obter o número, basta que o programa liste os fatores.

In [64]:
#function is_carmichael(primefaclist, counter = 1)
#    if counter == size(primefaclist)[1] + 1
#        return true
#    else
#        if ((reduce((x,y)->(x*y) % (primefaclist[counter]-1),primefaclist) - 1) % (primefaclist[counter]-1)) != 0
#            return false
#        else
#            return is_carmichael(primefaclist,counter+1)
#        end
#    end
#end
#
#,
#function carmichael_d_primos(d, primos)
#    if d == 1 || d == 2
#        return []
#    else
#        return [n for n in Combinatorics.Combinations(primos[2:end],d) if is_carmichael(n)]
#    end
#end

In [65]:
#function cap6exercicio9()
#    prims = erastotenes(10^3)
#    ANS = [carmichael_d_primos(d,prims[2:end]) for d in 1:8]
#    for x in enumerate(ANS)
#        print("Números d")
#        print(x)
#    end
#end
#@time cap6exercicio9()

In [66]:
#function list_carmichael(primos, prev=[], prev_p1=[])
#    if size(prev)[1] >= 8
#        return prev
#    else
#        if is_carmichael(prev)
#            ANS = []
#            for primo in primos
#                #part1 verificar se para todo prev_p1 ele é igual a 0
#                ((reduce((x,y)->(x*y) % (primefaclist[counter]-1),primefaclist) - 1) % (primefaclist[counter]-1)) != 0
#                for p_1 in prev_p1
#                    if p_1*primo % (prev)
#        else
#            return aux()
#    end
#end

In [67]:
#function teste(primos8)
#    counter = 0
#    ANS = []
#    for primos in primos8
#        push!(ANS,is_carmichael(primos))
#        counter += 1
#        if counter == 40000000
#            break
#        end
#    end
#    return ANS
#end

## Exercício 10

Escreva um programa para determinar o menor pseudoprimo forte para uma dada base. Vai ser necessário implementar o teste de Miller, de modo que a entrada seja um inteiro positivo $b>1$. O programa deve aplicar o teste de Miller na base $b$ aos ímpares compostos, até achar o primeiro número para qual o teste é inconclusivo. Uma maneira de se fazer isso, é programar o Crivo de Eratóstenes de maneira a conservar a lista dos ímpares compostos, em vez dos primos e depois testar as congruências. Aplique o teste para as bases $2$, $3$, $5$ e $7$. Quais os resultados obtidos?

In [68]:
function etapa_1(n::Int)
    n = n-1
    k = 0
    while (n%2 == 0)
        k += 1
        n = n/2
    end
    return n, k
end

function etapa_3(i::Int,r::Int,n::Int, k::Int)
   if((i == 0 && r == 1) || (i>= 0 && r == n-1)) 
        return false  ##inconclusivo!
    end
    #etapa 5
    if (i<k)
        #etapa 4
        return etapa_3(i+1,exercicio11(r,2,n),n,k)
    else
        return true ##composto
    end
end

function teste_miller_e_composto(n::Int,b::Int)
    ## verificar se os parametros sao validos
    if (n % 2 == 0 && (1 < b) && (b < n-1))
       throw(MathException(""))
    end
    
    ##teste de fato
    
    #etapa1
    q, k = etapa_1(n)
    
    #etapa2
    i, r = 0, exercicio11(b,q,n)
    
    #etapa 3
    return etapa_3(i,r,n,k)
    
end

teste_miller_e_composto (generic function with 1 method)

In [69]:
teste_miller_e_composto(341,2)

true

In [70]:
teste_miller_e_composto(25,2)

true

Revisão pro teste lógico
* **$p$ Primo** implica verdadeiro

* **$p$ composto** e **$q$ teste verdadeiro** implica verdadeiro

* **$p$ composto** e **$q$ teste falso** implica falso

* $\Rightarrow$ $p$ primo ou ($p$ composto e $q$ verdadeiro)

In [71]:
function exercicio10_cap6(b::Int64)::Int64
    cand, bigPrim = 3, 1000
    primos = erastotenes(bigPrim)
    while ((cand in primos) || (teste_miller_e_composto(cand,b) && !(cand in primos)))
        if (cand > bigPrim/2)
            bigPrim = bigPrim + 1000
            primos = erastotenes(bigPrim)
        end
        cand += 2
    end
    return cand
end

exercicio10_cap6 (generic function with 1 method)

In [72]:
exercicio10_cap6(2)

2047

In [73]:
exercicio10_cap6(3)

121

In [74]:
exercicio10_cap6(5)

781

In [75]:
exercicio10_cap6(7)

25